# Scrape ESPN Fantasy Football League

In order to scrape data from a private ESPN Fantasy Football League, we need to pass the SWID and the espn_s2 cookies values to the cookies parameter of the requests.get command.

In Chrome the SWID and the espn_s2 cookies values can be found here, chrome://settings/cookies/detail?site=espn.com.

To manually find the cookies values follow these instructions, settings > Cookies and other site data > All cookies and site data > espn.com > SWID/espn_s2.

Please note the cookies values differ from machine to machine.  My cookies values won't work on another comptuer.

<img src="SWID_cookie.PNG">

<img src="espn_s2_cookie.PNG">

Additionally, we need to pass the league id into the url.

To find the league id look in the url while logged into a private ESPN Fantasy Football league.

<img src="league_id.PNG">

#### Review Matchup URL:

To review any matchup:

Set teamId to the team id value (can be found below) and seasonId to 2020, 2019, or 2018.  ESPN's API v3 doesn't return data for years prior to 2018.

If the matchup under review is a **non-playoff matchup** then set both matchupPeriodId and scoringPeriodId to the same week number value.

If the matchup under review is a **playoff matchup** then the matchupPeriodId and scoringPeriodId week values are different.  For example, if we're reviewing a matchup during NFL weeks 15 and 16, which correspond to the last 2 playoff games in our league, then matchupPeriodId = 14 and scoringPeriodId = 15 or 16 for NFL weeks 15 or 16.

Example: https://fantasy.espn.com/football/boxscore?leagueId=169073&matchupPeriodId=1&scoringPeriodId=1&seasonId=2020&teamId=10&view=scoringperiod


#### <font color='red'>NOTE: Set user defined fields in cells with red titles</font>

In [1]:
# import needed packages
import numpy as np
import pandas as pd
import json
import os
import re, requests, bs4, csv, datetime
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# set pandas display options
pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',50)
pd.set_option('display.width',1000)

#### <font color='red'>Scrape ESPN Fantasy Football URL</font>

Source: https://stmorse.github.io/journal/espn-fantasy-v3.html

In [2]:
# set user define fields below: cookie values, league id, season, and current week number
swid      = '{F3E3A4D2-1203-4B11-BDAD-00C92AAA3F48}'
espn_s2   = 'AECf2TISP%2BRluNYIj24%2FqkuJBHTzAa7Br0BVIoUnBhHQCygnkN7hvyvU8wIm6XAT58otHnOZW4HjyImLJ14Rk8L9%2BwObWXeIa8kCosNFNMSc79r24KSiJK9jtqUQ2V1lvIncFM1qDhV9xL7E5jnutCP9rZfiJv8h%2Bq6WNvFb4YmyHICgx3QW68obk3wqBrGrXmw0vbq2bf367%2BsuL%2BOJ2ioRnxi1yY7LgbtiJnAXSKZj7kdIGjBiCPjUHgeXzDLHPunkLchEGlOrr%2FhG1ORgOsUX'
league_id = 169073
season    = 2021
week      = 12

# create url.  there are multiple views we can look at but we're interested in matchup data
url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/' + \
      str(season) + '/segments/0/leagues/' + str(league_id) + \
      '?view=mMatchup&view=mMatchupScore'

print(url)

# create JSON file for each week's matchup data
for i in range(1, week + 1):
    print('season: {}, week: {}'.format(season, i))
    r = requests.get(url,
                     params = {'scoringPeriodId': i},
                     cookies = {"SWID": swid, "espn_s2": espn_s2})
    d = r.json()
    with open('matchup_data/{}_matchups_week_{}.json'.format(season, i), 'w', encoding = 'utf-8') as f:
        json.dump(d, f, ensure_ascii = False, indent = 4)

https://fantasy.espn.com/apis/v3/games/ffl/seasons/2021/segments/0/leagues/169073?view=mMatchup&view=mMatchupScore
season: 2021, week: 1
season: 2021, week: 2
season: 2021, week: 3
season: 2021, week: 4
season: 2021, week: 5
season: 2021, week: 6
season: 2021, week: 7
season: 2021, week: 8
season: 2021, week: 9
season: 2021, week: 10
season: 2021, week: 11
season: 2021, week: 12


#### <font color='red'>Load JSON Files from Disk</font>

In [3]:
# open JSON files and save returned dictionaries to a list
year = 2021
week = 12
matchups_list = []

# load JSON file for each's week matchup data
for i in range(1, week + 1):
    f = open('matchup_data/{}_matchups_week_{}.json'.format(year, i))
    
    # returns JSON object as a dictionary 
    data = json.load(f)
    
    # add each JSON object to list created above
    matchups_list.append(data)

#### Create Dictionaries for Owner Names, Lineup Slot Names, Position Names, and NFL Team Names

Need player stat codes for:
* FG 60+ yards
* fumble recovered for td vs. fumble return for TD?

source for player stat codes: https://github.com/mkreiser/ESPN-Fantasy-Football-API/blob/master/src/player-stats/player-stats.js

In [4]:
# create dictionary of owner ids, owner team names, and owner names
owner_team_codes = {1:  ['Happy Rock Homewreckers', 'Blainer'],
                    2:  ["Bench Don't Kill My Vibe", 'Padge'],
                    4:  ['Seattle rainier riot', 'Boob'],
                    6:  ['Sticky Icky', 'T-$'],
                    7:  ['Springfield Atoms', 'Duvi'],
                    8:  ['Beacon Hill Posterizers', 'Bup'],
                    9:  ['Brookside Shokunin', 'Cheese'], 
                    10: ['CoMo FightinCamlToes', 'Doisy'],
                    11: ['Pixel Whippers','Sembower'],
                    15: ['Bud Lathrop Drive', 'Farmer']
}

# create dictionary of lineup slot ids and lineup names
lineup_slot_codes = {0:  'QB',
                     2:  'RB',
                     3:  'Flex',
                     4:  'WR',
                     6:  'TE', 
                     16: 'Def', 
                     17: 'K',
                     20: 'Bench', 
                     21: 'IR',
                     23: 'Flex'
}

# create dictionary of position ids and position names
position_codes = {1:  'QB',
                  2:  'RB',
                  3:  'WR',
                  4:  'TE',
                  5:  'KR',
                  16: 'DEF'
    
}

# create dictionary of team ids and team names
pro_team_codes = {0:  ['Free Agent', np.nan],
                  1:  ['Atlanta Falcons', 'ATL'],
                  2:  ['Buffalo Bills', 'BUF'],
                  3:  ['Chicago Bears', 'CHI'],
                  4:  ['Cincinnati Bengals', 'CIN'],
                  5:  ['Cleveland Browns', 'CLE'],
                  6:  ['Dallas Cowboys', 'DAL'],
                  7:  ['Denver Broncs', 'DEN'],
                  8:  ['Detroit Lions', 'DET'],
                  9:  ['Greenbay Packers', 'GB'],
                  10: ['Tennessee Titans', 'TEN'],
                  11: ['Indianapolis Colts', 'IND'],
                  12: ['Kansas City Chiefs', 'KC'],
                  13: ['Las Vegas Raiders', 'LV'],
                  14: ['Los Angeles Rams', 'LA'],
                  15: ['Miami Dolphins', 'MIA'],
                  16: ['Minnesota Vikings', 'MIN'],
                  17: ['New Engalnd Patriots', 'NE'],
                  18: ['New Orleans Saints', 'NO'],
                  19: ['New York Giants', 'NYG'],
                  20: ['New York Jets', 'NYJ'],
                  21: ['Philadelphia Eagles', 'PHI'],
                  22: ['Arizona Cardinals', 'ARI'],
                  23: ['Pittsburgh Steelers', 'PIT'],
                  24: ['Los Angeles Chargers', 'LAC'],
                  25: ['San Francisco 49ers', 'SF'],
                  26: ['Seattle Seahawks', 'SEA'],
                  27: ['Tampa Bay Buccaneers', 'TB'],
                  28: ['Washington Football Team', 'WAS'],
                  29: ['Carolina Panthers', 'CAR'],
                  30: ['Jacksonville Jaguars', 'JAX'],
                  33: ['Baltimore Ravens', 'BAL'],
                  34: ['Houston Texans', 'HOU']
}

# create dictionary of real game statistics codes
player_stat_codes = {0:   'pass_att',
                     1:   'pass_comp',
                     2:   'pass_incomp',
                     3:   'pass_yrd',
                     4:   'pass_td',
                     5:   'pass_5_yrd',
                     6:   'unk6',
                     7:   'unk7',
                     8:   'unk8',
                     9:   'unk9',
                     10:  'unk10',
                     11:  'unk11',
                     12:  'unk12',
                     13:  'unk13',
                     14:  'unk14',
                     15:  'unk15',
                     16:  'pass_50_yrd_td',
                     17:  'pass_yrd_300_399',
                     18:  'pass_yrd_400+',
                     19:  'unk19',
                     19:  'pass_2pt_con',
                     20:  'pass_int',
                     21:  'unk21',
                     22:  'pass_yrd_dupe',
                     23:  'rush_att',
                     24:  'rush_yrd',
                     25:  'rush_td',
                     26:  'rush_2pt_con',
                     27:  'rush_5_yrd',
                     28:  'unk28',
                     29:  'unk29',
                     30:  'unk30',
                     31:  'unk31',
                     32:  'unk32',
                     33:  'unk33',
                     34:  'unk34',
                     35:  'unk35',
                     36:  'rush_50_yrd_td',
                     37:  'rush_yrd_100_199',
                     38:  'rush_yrd_200+',
                     39:  'unk39',
                     40:  'unk40',
                     41:  'receptions_dupe',
                     42:  'rec_yrd',
                     43:  'rec_td',
                     44:  'rec_2pt_con',
                     45:  'unk45',
                     46:  'rec_50_yrd_td',
                     47:  'rec_5_yrd',
                     48:  'unk48',
                     49:  'unk49',
                     50:  'unk50',
                     51:  'unk51',
                     52:  'unk52',
                     53:  'receptions',
                     54:  'unk54',
                     55:  'unk55',
                     56:  'rec_yrd_100_199',
                     57:  'rec_yrd_200+',
                     58:  'rec_tar',
                     59:  'yac',
                     60:  'yrd_per_rec',
                     61:  'rec_yrd_dupe',
                     62:  'unk62',
                     64:  'unk64',
                     65:  'unk65',
                     66:  'unk66',
                     67:  'unk67',
                     68:  'unk68',
                     69:  'unk69',
                     70:  'unk70',
                     71:  'unk71',
                     72:  'fum_lost',
                     73:  'unk73',
                     74:  'fg_made_50+',
                     75:  'unk75',
                     76:  'unk76',
                     77:  'fg_made_40_49',
                     78:  'unk78',
                     79:  'fg_miss_40_49',
                     80:  'fg_made_0_39',
                     81:  'unk81',
                     82:  'fg_miss_0_39',
                     83:  'fg_con',
                     84:  'fg_att',  
                     85:  'fg_miss_tot',
                     86:  'pat_con',
                     87:  'pat_att',
                     88:  'pat_miss_tot',
                     89:  'def_st_0_pts_alw',
                     90:  'def_st_1_6_pts_alw',
                     91:  'def_st_7_13_pts_alw',
                     92:  'def_st_14_17_pts_alw',
                     93:  'def_st_blk_td',
                     94:  'unk94',
                     95:  'def_st_int',
                     96:  'def_st_fum',
                     97:  'def_st_blk_kick',
                     98:  'def_st_safety',
                     99:  'def_st_sack',
                     100: 'unk100',
                     101: 'def_st_kick_ret_td',
                     102: 'def_st_punt_ret_td',
                     103: 'def_st_int_td',
                     104: 'def_st_fum_ret_td',
                     105: 'unk105',
                     106: 'unk106',
                     107: 'unk107',
                     108: 'unk108',
                     109: 'unk109',
                     110: 'unk110',
                     111: 'unk111',
                     112: 'unk112',
                     113: 'unk113',
                     114: 'unk114',
                     115: 'unk115',
                     116: 'unk116',
                     117: 'unk117',
                     118: 'unk118',
                     119: 'unk119',
                     120: 'def_pts_alw',
                     121: 'unk121',
                     122: 'def_st_22_27_pts_alw',
                     123: 'def_st_28_34_pts_alw',
                     124: 'def_st_35_45_pts_alw',
                     125: 'def_st_46+_pts_alw',
                     127: 'def_tot_yrd_alw',
                     128: 'def_st_0_99_yrd_alw',
                     129: 'def_st_100_199_yrd_alw',
                     130: 'def_st_200_299_yrd_alw',
                     131: 'unk131',
                     132: 'def_st_350_399_yrd_alw',
                     133: 'def_st_400_449_yrd_alw',
                     134: 'def_st_450_499_yrd_alw',
                     135: 'def_st_500_549_yrd_alw',
                     136: 'def_st_550+_yrd_alw',
                     155: 'unk155',
                     156: 'unk156',
                     158: 'unk158',
                     175: 'unk175',
                     176: 'unk176',
                     177: 'unk177',
                     178: 'unk178',
                     179: 'unk179',
                     180: 'unk180',
                     181: 'unk181',
                     182: 'unk182',
                     183: 'unk183',
                     184: 'unk184',
                     185: 'unk185',
                     186: 'unk186',
                     187: 'unk187',
                     188: 'unk188',
                     189: 'unk189',
                     190: 'unk190',
                     191: 'unk191',
                     192: 'unk192',
                     193: 'unk193',
                     194: 'unk194',
                     195: 'unk195',
                     196: 'unk196',
                     197: 'unk197',
                     198: 'fg_made_50_59',
                     199: 'unk199',
                     200: 'unk200',
                     202: 'unk202',
                     203: 'unk203',
                     210: 'unk210',
}

# create dictionary of fantasy football specific statistics codes
ff_scoring_codes = {1:   'pass_comp_ff',
                    2:   'pass_incomp_ff',
                    4:   'pass_td_ff',
                    5:   'pass_5_yrd_ff',
                    16:  'pass_50_yrd_td_ff',
                    17:  'pass_yrd_300_399_ff',
                    18:  'pass_yrd_400+_ff',
                    19:  'pass_2pt_con_ff',
                    20:  'pass_int_ff',
                    25:  'rush_td_ff',
                    26:  'rush_2pt_con_ff',
                    27:  'rush_5_yrd_ff',
                    36:  'rush_50_yrd_td_ff',
                    37:  'rush_yrd_100_199_ff',
                    38:  'rush_yrd_200+_ff',
                    43:  'rec_td_ff',
                    44:  'rec_2pt_con_ff_ff',
                    46:  'rec_50_yrd_td_ff',
                    47:  'rec_5_yrd_ff',
                    53:  'receptions_ff',
                    56:  'rec_yrd_100_199_ff',
                    57:  'rec_yrd_200+_ff',
                    72:  'fum_lost_ff',
                    77:  'fg_made_40_49_ff',
                    79:  'fg_miss_40_49_ff',
                    80:  'fg_made_0_39_ff',
                    82:  'fg_miss_0_39_ff',
                    86:  'pat_made_ff',
                    88:  'pat_miss_ff',
                    89:  'def_st_0_pts_alw_ff',
                    90:  'def_st_1_6_pts_alw_ff',
                    91:  'def_st_7_13_pts_alw_ff',
                    92:  'def_st_14_17_pts_alw_ff',
                    93:  'def_st_blk_td_ff',
                    95:  'def_st_int_ff',
                    96:  'def_st_fum_ff',
                    97:  'def_st_blk_kick_ff',
                    98:  'def_st_safety_ff',
                    99:  'def_st_sack_ff',
                    101: 'def_st_kick_ret_td_ff',
                    102: 'def_st_punt_ret_td_ff',
                    103: 'def_st_int_td_ff',
                    104: 'def_st_fum_ret_td_ff',
                    122: 'def_st_22_27_pts_alw_ff',
                    123: 'def_st_28_34_pts_alw_ff',
                    124: 'def_st_35_45_pts_alw_ff',
                    125: 'def_st_46+_pts_alw_ff',
                    128: 'def_st_0_99_yrd_alw_ff',
                    129: 'def_st_100_199_yrd_alw_ff',
                    130: 'def_st_200_299_yrd_alw_ff',
                    132: 'def_st_350_399_yrd_alw_ff',
                    133: 'def_st_400_449_yrd_alw_ff',
                    134: 'def_st_450_499_yrd_alw_ff',
                    135: 'def_st_500_549_yrd_alw_ff',
                    136: 'def_st_550+_yrd_alw_ff',
                    198: 'fg_made_50_59_ff'
}

# create dictionary of fantasy football scoring values
scoring_dict = {'pass_5_yrd':            0.1,
                'pass_comp':             0.4,
                'pass_incomp':           -0.2,
                'pass_td':                6,
                'pass_50_yrd_td':         3,
                'pass_int':               -2,
                'pass_2pt_con':           2,
                'pass_yrd_300_399':       3,
                'P400':                   5,
                'rush_5_yrd':             0.6,
                'rush_td':                6,
                'RTD50':                  3,
                'rush_2pt_con':           2,
                'rush_yrd_100_199':       3,
                'RY200':                  5,
                'rec_5_yrd':              0.6,
                'receptions':             1,
                'rec_td':                 6,
                'rec_50_yrd_td':          3,
                'rec_2pt_con':            2,
                'rec_yrd_100_199':        3,
                'REY200':                 5,
                'pat_made':               1,
                'pat_miss':               -1,
                'fg_made_0_39':           3,
                'fg_made_40_49':          4,
                'fg_miss_0_39':           -2,
                'fg_miss_40_49':          -1,
                'fg_made_50':             5,
                'FG60':                   5,
                'def_st_kick_ret_td':     6,
                'def_st_punt_ret_td':     6,
                'def_st_int_td':          5,
                'def_st_fum_ret_td':      5,
                'def_st_blk_td':          6,
                'def_st_sack':            1,
                'def_st_blk_kick':        2,
                'def_st_int':             3,
                'def_st_fum':             3,
                'def_st_safety':          2,
                'def_st_0_pts_alw':       10,
                'def_st_1_6_pts_alw':     7,
                'def_st_7_13_pts_alw':    3,
                'def_st_14_17_pts_alw':   1,
                'def_st_22_27_pts_alw':   -1,
                'def_st_28_34_pts_alw':   -3,
                'def_st_35_45_pts_alw':   -5,
                'PA46':                   -7,
                'def_st_0_99_yrd_alw':    7,
                'def_st_100_199_yrd_alw': 3,
                'def_st_200_299_yrd_alw': 1,
                'def_st_400_449_yrd_alw': -1,
                'def_st_450_499_yrd_alw': -1.5,
                'def_st_500_549_yrd_alw': -2,
                'def_st_550+_yrd_alw':    -3,
                'misc_kick_ret_td':       6,
                'misc_punt_ret_td':       6,
                'misc_fum_rec_td':        6,
                'misc_fum_lost':          -2,
                'misc_fum_ret_td':        6
}

#### Create Rosters Dataframe per Team per Week

In [5]:
# initialize list needed to create rosters_df
data_list = []

# loop through each JSON object in matchups_list which represents one week's matchup data
for wk in range(0, len(matchups_list)):
    #print(wk)
    
    # grab year
    year = matchups_list[wk]['seasonId']
    
    # loop through each team
    for tm in matchups_list[wk]['teams']:
        owner_team_id   = tm['id']
        owner_team_name = owner_team_codes[owner_team_id][0]
        owner_name      = owner_team_codes[owner_team_id][1]
        #print(owner_team_name)
    
        # loop through weekly roster
        for p in tm['roster']['entries']:
            # grab week number
            temp_week = matchups_list[wk]['scoringPeriodId']
            
            # extract roster data
            player_name   = p['playerPoolEntry']['player']['fullName']
            #print(player_name)
            slot_id       = p['lineupSlotId']
            #print(slot_id)
            slot_name     = lineup_slot_codes[slot_id]
            position_id   = p['playerPoolEntry']['player']['defaultPositionId']
            position_name = position_codes[position_id]

            # injured status (need try/exc bc of D/ST)
            current_inj = np.nan
            try:
                current_inj = p['playerPoolEntry']['player']['injuryStatus']
            except:
                pass

            # projected/actual points
            # note:  need to grab team data in different spot within the JSON object since that data isn't alway
            # in the same spot.  data integrity issue
            proj_points, actual_points = None, None
            for stat in p['playerPoolEntry']['player']['stats']:
                if stat['scoringPeriodId'] != temp_week:
                    continue
                if stat['statSourceId'] == 0:
                    actual_points = stat['appliedTotal']
                    if stat['proTeamId'] != 0:
                        
                        # grab team info
                        pro_team_id = stat['proTeamId']
                        pro_team_name = pro_team_codes[pro_team_id][0]
                        pro_team_name_abv = pro_team_codes[pro_team_id][1]                        
                elif stat['statSourceId'] == 1:
                    proj_points = stat['appliedTotal']
                    if stat['proTeamId'] != 0:
                        
                        # grab team info
                        pro_team_id = stat['proTeamId']
                        pro_team_name = pro_team_codes[pro_team_id][0]
                        pro_team_name_abv = pro_team_codes[pro_team_id][1]
                    elif (proj_points < 1) & (not actual_points):
                        
                        # grab team info
                        pro_team_id   = p['playerPoolEntry']['player']['proTeamId']
                        pro_team_name = pro_team_codes[pro_team_id][0]
                        pro_team_name_abv = pro_team_codes[pro_team_id][1] 
                    elif not pro_team_id:
                        
                        # grab team info
                        pro_team_id   = p['playerPoolEntry']['player']['proTeamId']
                        pro_team_name = pro_team_codes[pro_team_id][0]
                        pro_team_name_abv = pro_team_codes[pro_team_id][1]                       
            
            # add data to list created above
            data_list.append([year, temp_week, owner_team_name, owner_name, player_name, pro_team_name, pro_team_name_abv, 
                              current_inj, slot_name, position_name, proj_points, actual_points, slot_id])

# create rosters_df using data_list
rosters_df = pd.DataFrame(data_list, 
                          columns=['year', 'week', 'owner_team', 'owner', 'player', 'pro_team', 'pro_team_abv',
                                   'current_inj_status', 'lineup_slot_name', 'position_name', 'proj_points', 
                                   'actual_points', 'slot_id'
                                  ]
                         )

# save list of column names for later use
rosters_df_columns = rosters_df.columns.tolist()

# save to csv
rosters_df.to_csv(f"rosters_df_{year}.csv", index = False)

# # filter by 2020 season
# rosters_df_2020 = rosters_df.loc[(rosters_df['year'] == 2020)]

# # save to csv
# rosters_df_2020.to_csv("rosters_df_2020.csv", index = False)

# explore data frame
rosters_df.info()
rosters_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   year                2400 non-null   int64  
 1   week                2400 non-null   int64  
 2   owner_team          2400 non-null   object 
 3   owner               2400 non-null   object 
 4   player              2400 non-null   object 
 5   pro_team            2400 non-null   object 
 6   pro_team_abv        2399 non-null   object 
 7   current_inj_status  2218 non-null   object 
 8   lineup_slot_name    2400 non-null   object 
 9   position_name       2400 non-null   object 
 10  proj_points         2388 non-null   float64
 11  actual_points       2259 non-null   float64
 12  slot_id             2400 non-null   int64  
dtypes: float64(2), int64(3), object(8)
memory usage: 243.9+ KB


,year,week,proj_points,actual_points,slot_id
count,2400.0,2400.000000,2388.000000,2259.000000,2400.000000
mean,2021.0,6.500000,11.816230,12.209252,13.905833
std,0.0,3.452772,7.711025,10.653760,8.068866
min,2021.0,1.000000,0.000000,-6.500000,0.000000
25%,2021.0,3.750000,7.330560,4.000000,4.000000
50%,2021.0,6.500000,11.342226,10.000000,20.000000
75%,2021.0,9.250000,16.587636,18.050000,20.000000
max,2021.0,12.000000,36.933048,60.600000,23.000000


#### Add Fantasy Scoring Data to Rosters Dataframe

In [7]:
# create pandas dataframe using the fantasy football scoring dictionary's values as column names
ff_scoring_df = pd.DataFrame(np.zeros((len(rosters_df), len(ff_scoring_codes.values())))
                            ,columns = list(ff_scoring_codes.values()))

# create pandas dataframe using the player stat dictionary's values as column names
player_stat_df = pd.DataFrame(np.zeros((len(rosters_df), len(player_stat_codes.values())))
                             ,columns = list(player_stat_codes.values()))

# combine dataframe created above to the rosters dataframe
rosters_df = pd.concat([rosters_df, ff_scoring_df, player_stat_df], axis = 1)
print(rosters_df.shape)

(1200, 236)


In [6]:
# loop through each JSON object in matchups_list which represents one week's matchup data
for wk in range(0, len(matchups_list)):
    
    # grab year
    year = matchups_list[wk]['seasonId']

    # loop through each team
    for tm in matchups_list[wk]['teams']:

        # loop through weekly roster
        for p in tm['roster']['entries']:
            #temp_week = wk + 1
            temp_week = matchups_list[wk]['scoringPeriodId']
            
            # grab player name
            player_name = p['playerPoolEntry']['player']['fullName']

            # loop through each set of stats
            for stat in p['playerPoolEntry']['player']['stats']:
                if stat['scoringPeriodId'] != temp_week:
                    continue
                if stat['statSourceId'] == 0:
                    
                    # loop through the fantasy scoring stats
                    for i in [int(s) for s in stat['appliedStats'].keys()]:

                        # if the scoring code exists in the dictionary above then add the stat to rosters_df
                        if i in ff_scoring_codes.keys():
                            rosters_df.loc[(rosters_df['player'] == player_name) & (rosters_df['week'] == temp_week) &\
                                           (rosters_df['year'] == year), ff_scoring_codes[i]] = stat['appliedStats'][str(i)]

                    # loop through the player stats
                    for j in [int(r) for r in stat['stats'].keys()]:

                        # if the scoring code exists in the dictionary above then add the stat to rosters_df
                        if j in player_stat_codes.keys():
                            rosters_df.loc[(rosters_df['player'] == player_name) & (rosters_df['week'] == temp_week) &\
                                           (rosters_df['year'] == year), player_stat_codes[j]] = stat['stats'][str(j)]

# replace nulls with 0
rosters_df.replace(np.nan, 0, inplace=True)

# save to csv
rosters_df.to_csv(f"rosters_df_w_scoring_{year}.csv", index = False)

# explore data frame
rosters_df.info()
rosters_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Columns: 224 entries, year to rec_yrd_200+
dtypes: float64(213), int64(3), object(8)
memory usage: 4.1+ MB


,year,week,proj_points,actual_points,slot_id,rush_td_ff,rush_5_yrd_ff,rush_att,rush_yrd,rush_td,rush_5_yrd,unk28,unk29,unk30,unk31,unk33,unk34,unk39,unk40,unk155,unk158,unk179,unk210,receptions_ff,receptions_dupe,rec_yrd,receptions,rec_tar,yac,yrd_per_rec,rec_yrd_dupe,unk156,rec_yrd_100_199_ff,rec_td_ff,rec_5_yrd_ff,rec_td,rec_5_yrd,unk48,unk49,unk50,unk51,unk52,unk54,rec_yrd_100_199,unk115,unk118,unk185,unk108,unk109,pass_comp_ff,pass_incomp_ff,pass_int_ff,pass_5_yrd_ff,pass_att,pass_comp,pass_incomp,pass_yrd,pass_5_yrd,unk6,unk7,unk8,unk9,unk10,unk11,unk12,unk13,unk14,pass_int,unk21,pass_yrd_dupe,unk64,unk73,rec_50_yrd_td_ff,unk45,rec_50_yrd_td,unk183,def_st_0_99_yrd_alw_ff,def_st_fum_ff,def_st_100_199_yrd_alw_ff,def_st_blk_kick_ff,def_st_200_299_yrd_alw_ff,def_st_sack_ff,def_st_400_449_yrd_alw_ff,def_st_450_499_yrd_alw_ff,def_st_500_549_yrd_alw_ff,def_st_550+_yrd_alw_ff,def_st_0_pts_alw_ff,def_st_22_27_pts_alw_ff,def_st_1_6_pts_alw_ff,def_st_28_34_pts_alw_ff,def_st_7_13_pts_alw_ff,def_st_35_45_pts_alw_ff,def_st_14_17_pts_alw_ff,def_st_blk_td_ff,def_st_0_pts_alw,def_st_1_6_pts_alw,def_st_7_13_pts_alw,def_st_14_17_pts_alw,def_st_blk_td,def_st_fum,def_st_blk_kick,def_st_sack,unk100,unk105,unk106,unk107,unk110,unk111,unk112,unk113,def_pts_alw,unk121,def_st_22_27_pts_alw,def_st_28_34_pts_alw,def_st_35_45_pts_alw,def_tot_yrd_alw,def_st_0_99_yrd_alw,def_st_100_199_yrd_alw,def_st_200_299_yrd_alw,unk131,def_st_350_399_yrd_alw,def_st_400_449_yrd_alw,def_st_450_499_yrd_alw,def_st_500_549_yrd_alw,def_st_550+_yrd_alw,unk187,unk188,unk189,unk190,unk191,unk192,unk193,unk194,unk195,unk65,unk68,unk184,fg_made_0_39_ff,pat_made_ff,fg_made_50_59_ff,fg_made_50+,unk75,fg_made_0_39,unk81,fg_con,fg_att,pat_con,pat_att,fg_made_50_59,unk199,unk55,pass_50_yrd_td_ff,pass_yrd_300_399_ff,pass_td_ff,pass_td,unk15,pass_50_yrd_td,pass_yrd_300_399,unk175,unk176,rush_50_yrd_td_ff,rush_yrd_100_199_ff,unk32,unk35,rush_50_yrd_td,rush_yrd_100_199,unk114,unk116,unk117,def_st_int_td_ff,def_st_int_ff,unk94,def_st_int,def_st_int_td,fum_lost_ff,unk66,unk70,fum_lost,unk177,pass_yrd_400+_ff,pass_yrd_400+,unk67,unk71,rush_2pt_con_ff,rush_2pt_con,unk69,unk180,fg_made_40_49_ff,fg_made_40_49,unk78,rec_2pt_con_ff_ff,rec_2pt_con,unk182,fg_miss_0_39_ff,pat_miss_ff,unk76,fg_miss_0_39,fg_miss_tot,pat_miss_tot,unk202,unk203,unk119,fg_miss_40_49_ff,fg_miss_40_49,def_st_fum_ret_td_ff,def_st_fum_ret_td,unk181,unk178,unk200,pass_2pt_con_ff,pass_2pt_con,unk186,def_st_kick_ret_td_ff,def_st_kick_ret_td,rec_yrd_200+_ff,rec_yrd_200+
count,2400.0,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.00000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.00000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000

#### Create Matchups Dataframe per Team per Week

In [7]:
# initialize list needed to create matchups_df
data_list = []

# loop through each matchup from week 1 to current week
for i in range(0, len(data['schedule'])):
    
    if data['schedule'][i]['away']['totalPoints'] == 0:
        break
        
    # build row for away team
    away_week = data['schedule'][i]['matchupPeriodId']
    away_owner_team_id = data['schedule'][i]['away']['teamId']
    away_owner_team_name = owner_team_codes[away_owner_team_id][0]
    away_owner_name = owner_team_codes[away_owner_team_id][1]
    away_score = data['schedule'][i]['away']['totalPoints']
    away_opp_id = data['schedule'][i]['home']['teamId']
    away_opp_team_name = owner_team_codes[away_opp_id][0]
    away_opp_name = owner_team_codes[away_opp_id][1]
    away_opp_score = data['schedule'][i]['home']['totalPoints']
    
    # determine if away team won
    if data['schedule'][i]['winner'] == 'AWAY':
        away_win = 1
    else:
        away_win = 0
    # append away row to data_list
    data_list.append([away_week, away_owner_team_name, away_owner_name, away_score, away_win, away_opp_team_name, 
                      away_opp_name, away_opp_score])
    
    # build row for home team
    home_week = data['schedule'][i]['matchupPeriodId']
    home_owner_team_id = data['schedule'][i]['home']['teamId']
    home_owner_team_name = owner_team_codes[home_owner_team_id][0]
    home_owner_name = owner_team_codes[home_owner_team_id][1]
    home_score = data['schedule'][i]['home']['totalPoints']
    home_opp_id = data['schedule'][i]['away']['teamId']
    home_opp_team_name = owner_team_codes[home_opp_id][0]
    home_opp_name = owner_team_codes[home_opp_id][1]
    home_opp_score = data['schedule'][i]['away']['totalPoints']
    
    # determine if home team won    
    if data['schedule'][i]['winner'] == 'HOME':
        home_win = 1
    else:
        home_win = 0
    
    # append home row to data_list
    data_list.append([home_week, home_owner_team_name, home_owner_name, home_score, home_win, home_opp_team_name, 
                      home_opp_name, home_opp_score])

# create matchups_df using data_list
matchups_df = pd.DataFrame(data_list, 
                          columns=['week', 'owner_team_name', 'owner', 'score', 'win', 'opp_owner_team_name', 'opp_owner', 
                                   'opp_score'])
# save to csv
matchups_df.to_csv(f"matchups_df_{year}.csv", index = False)

# explore data frame
matchups_df.info()
matchups_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   week                 120 non-null    int64  
 1   owner_team_name      120 non-null    object 
 2   owner                120 non-null    object 
 3   score                120 non-null    float64
 4   win                  120 non-null    int64  
 5   opp_owner_team_name  120 non-null    object 
 6   opp_owner            120 non-null    object 
 7   opp_score            120 non-null    float64
dtypes: float64(2), int64(2), object(4)
memory usage: 7.6+ KB


,week,score,win,opp_score
count,120.000000,120.000000,120.000000,120.000000
mean,6.500000,155.135833,0.500000,155.135833
std,3.466527,28.802707,0.502096,28.802707
min,1.000000,56.700000,0.000000,56.700000
25%,3.750000,134.500000,0.000000,134.500000
50%,6.500000,156.900000,0.500000,156.900000
75%,9.250000,173.150000,1.000000,173.150000
max,12.000000,228.900000,1.000000,228.900000


In [8]:
# subset matchups_df by wins
wins = matchups_df.loc[matchups_df['win'] == 1]

# create total_wins dataframe of wins per team
total_wins = pd.DataFrame(wins.groupby(['owner_team_name'])['win'].value_counts().reset_index(0).reset_index(drop=True))
total_wins.columns = ['owner_team_name', 'wins']

# subset matchups_df by losses
losses = matchups_df.loc[matchups_df['win'] == 0]

# create total_losses dataframe of losses per team
total_losses = pd.DataFrame(losses.groupby(['owner_team_name'])['win'].value_counts().reset_index(0).reset_index(drop=True))
total_losses.columns = ['owner_team_name', 'losses']

# merge total_wins and total_losses
win_loss_df = total_wins.merge(total_losses, on = 'owner_team_name', how = 'left')

# replace any null values with 0 which means one or more teams have either 0 wins or 0 losses
win_loss_df.fillna(0, inplace=True)

# fillna function casts dtype to float so change dtype back to int
win_loss_df['losses'] = win_loss_df['losses'].astype('int')
win_loss_df['wins'] = win_loss_df['wins'].astype('int')

# # create total_points dataframe of wins per team
total_points_df = pd.DataFrame(matchups_df.groupby(['owner_team_name'])[['score', 
                                                                     'opp_score']].sum().reset_index(0).reset_index(drop=True))
total_points_df.columns = ['owner_team_name', 'points_for', 'points_against']

# merge win_loss_df with total_points_df
win_loss_df = win_loss_df.merge(total_points_df, on = 'owner_team_name', how = 'left')

# save to csv
win_loss_df.to_csv(f"win_loss_df_{year}.csv", index = False)

# explore data frame
win_loss_df.info()
win_loss_df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   owner_team_name  10 non-null     object 
 1   wins             10 non-null     int32  
 2   losses           10 non-null     int32  
 3   points_for       10 non-null     float64
 4   points_against   10 non-null     float64
dtypes: float64(2), int32(2), object(1)
memory usage: 400.0+ bytes


,wins,losses,points_for,points_against
count,10.000000,10.000000,10.00000,10.000000
mean,6.000000,6.000000,1861.63000,1861.630000
std,1.763834,1.763834,112.83197,115.401137
min,4.000000,3.000000,1736.10000,1706.800000
25%,5.000000,4.500000,1758.17500,1780.125000
50%,5.500000,6.500000,1835.25000,1855.250000
75%,7.500000,7.000000,1977.07500,1927.475000
max,9.000000,8.000000,2015.30000,2047.000000
